In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import pymorphy2

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`.

In [ ]:
text = '''с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий'''

In [ ]:
import Levenshtein  # библиотека для расчета расстояния Левенштейна

# читаем файл и записываем все слова в список
with open("litw-win.txt", "r", encoding="utf-8", errors="ignore") as file:
    words = file.read().split()


# заданное предложение
sentence = "Ths book is very goood."

# разбиваем предложение на слова и исправляем опечатки
corrected_sentence = []
for word in sentence.split():
    if word in words:  # если слово есть в списке words, то оставляем его без изменений
        corrected_sentence.append(word)
    else:  # иначе ищем ближайшее слово из списка words
        closest_word = min(words, key=lambda x: Levenshtein.distance(word, x))
        corrected_sentence.append(closest_word)

# объединяем слова в исправленное предложение
corrected_sentence = " ".join(corrected_sentence)

print(corrected_sentence)

2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.

In [ ]:
import nltk
nltk.download('punkt')
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer

# разбиваем текст на слова
text = '''с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий'''
words = nltk.word_tokenize(text, language='russian')

# стемминг слов
stemmer = SnowballStemmer("russian")
stemmed_words = [stemmer.stem(word) for word in words]

# лемматизация слов
lemmatizer = WordNetLemmatizer()
lemmatized_words = [lemmatizer.lemmatize(word) for word in words]

print("Слова:", words)
print("Стемминг:", stemmed_words)
print("Лемматизация:", lemmatized_words)

3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи CountVectorizer.

# *** Лабораторная работа 9**

**Расстояние редактирования**

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [6]:
import pandas as pd
import nltk
nltk.download('punkt')

data = pd.read_csv('preprocessed_descriptions.csv')

words = set()
for description in data['preprocessed_descriptions']:
    if pd.notnull(description): # check for null values
        if not isinstance(description, str):
            description = str(description) # convert non-string values to strings
        words.update(nltk.word_tokenize(description))

[nltk_data] Downloading package punkt to /home/datalore/nltk_data...
[nltk_data]   Package punkt is already up-to-date!



1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [3]:
import random
import editdistance

# загрузка файла preprocessed_descriptions.csv
with open('preprocessed_descriptions.csv', 'r') as f:
    words = f.read().split()

# генерация 5 пар случайных слов из списка
pairs = [(random.choice(words), random.choice(words)) for i in range(5)]

# рассчет расстояния редактирования для каждой пары слов
for w1, w2 in pairs:
    distance = editdistance.eval(w1, w2)
    print(f'Расстояние между словами "{w1}" и "{w2}": {distance}')

Расстояние между словами "as" и "rolls": 4
Расстояние между словами "threads" и "my": 7
Расстояние между словами "this" и "this": 0
Расстояние между словами "carney" и "a": 5
Расстояние между словами "recipes" и "comes": 4


1.3 Напишите функцию, которая для заданного слова word возвращает k ближайших к нему слов из списка words (близость слов измеряется с помощью расстояния Левенштейна)

In [4]:
from Levenshtein import distance
import heapq


def k_nearest_words(word, words, k):
    # создаем список пар (расстояние Левенштейна, слово)
    distances = [(distance(word, w), w) for w in words]
    # используем heapq для нахождения k элементов с наименьшим расстоянием
    k_nearest = heapq.nsmallest(k, distances, key=lambda x: x[0])
    # возвращаем только слова
    return [w for _, w in k_nearest]

- **Стемминг, лемматизация**

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

**Векторное представление текста**

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи TfidfVectorizer

In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

recipes = pd.read_csv('preprocessed_descriptions.csv', encoding='utf-8')

# choose 5 random recipes
random_recipes = recipes.sample(n=5)

# transform recipe descriptions into vectors using TfidfVectorizer
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(random_recipes['preprocessed_descriptions'])

# create dataframe with recipe names and their vectors
recipe_vectors = pd.DataFrame(vectors.toarray(), columns=vectorizer.get_feature_names())
recipe_vectors.index = random_recipes['name']

AttributeError: AttributeError: 'TfidfVectorizer' object has no attribute 'get_feature_names'

In [18]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

recipes = pd.read_csv('preprocessed_descriptions.csv', encoding='utf-8')

random_recipes = recipes.sample(n=5)


vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(random_recipes['preprocessed_descriptions'])

recipe_vectors = pd.DataFrame(vectors.toarray(), columns=vectorizer.get_feature_names_out())
recipe_vectors.index = random_recipes['name']
print(recipe_vectors.index)


Index(['cornbread  wild mushroom  and rice stuffing',
       'mock hummus bi tahini  chickpea   sesame seed  paste',
       'pacific northwest stir fried asparagus   sugar snap peas',
       'ground beef  wellington  with fennel',
       'cheerwine holiday party punch'],
      dtype='object', name='name')


3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

In [19]:
from scipy.spatial.distance import cosine
import numpy as np

# calculate cosine distance between recipe vectors
cosine_distances = np.zeros((5, 5))
for i in range(5):
    for j in range(5):
        cosine_distances[i][j] = cosine(recipe_vectors.iloc[i], recipe_vectors.iloc[j])

# create dataframe with recipe names and their cosine distances
cosine_distances_df = pd.DataFrame(cosine_distances, columns=random_recipes['name'], index=random_recipes['name'])
print(cosine_distances_df)

name                                                cornbread  wild mushroom  and rice stuffing  \
name                                                                                              
cornbread  wild mushroom  and rice stuffing                                                 0.0   
mock hummus bi tahini  chickpea   sesame seed  ...                                          1.0   
pacific northwest stir fried asparagus   sugar ...                                          1.0   
ground beef  wellington  with fennel                                                        1.0   
cheerwine holiday party punch                                                               1.0   

name                                                mock hummus bi tahini  chickpea   sesame seed  paste  \
name                                                                                                       
cornbread  wild mushroom  and rice stuffing                                               

3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).